In [ ]:
import os
import cv2
import numpy as np

#Encoding and Split data into Train/Test Sets
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split as tts

#Tensorflow Keras CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.applications import ResNet50

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD

from sklearn.model_selection import KFold

#Plot Images
import matplotlib.pyplot as plt

In [ ]:
data = []
label = []

for folder in os.listdir('../input/plant-seedlings-classification/train'):
    for file in os.listdir(os.path.join('../input/plant-seedlings-classification/train', folder)):
        label.append(folder)
        img = cv2.imread(os.path.join('../input/plant-seedlings-classification/train', folder, file))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        im = cv2.resize(img_rgb, (128,128))
        data.append(im)

In [ ]:
data = np.array(data)
label = np.array(label)

In [ ]:
X = data / 255

In [ ]:
encoder = LabelEncoder()

In [ ]:
y = encoder.fit_transform(label)
y = to_categorical(y, 12)

In [ ]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

## Convolutional Neural Network

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores = []
histories = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (128,128,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(12, activation = "softmax"))

    opt = Adam(learning_rate=0.01)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit(X[train], y[train],
                batch_size=25,epochs=50,
                verbose=1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## Dense Neural Network

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_d = []
histories_d = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(200))
    model.add(Dense(100))
    model.add(Dense(64))
    model.add(Dense(32)) 
    model.add(Dense(24))
    model.add(Dense(16))
    model.add(Dense(12, activation='softmax'))

    opt = Adam(learning_rate=0.01)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit(X[train], y[train],
                batch_size=25,epochs=50,
                verbose=1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## Alex Net

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_a = []
histories_a = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(128,128,3)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.25)),
    model.add(Dense(12, activation='softmax'))
    
    opt = Adam(learning_rate=0.01)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit(X[train], y[train],
                batch_size=25,epochs=50,
                verbose=1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## ResNet

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_res = []
histories_res = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
    model.add(Dense(64))
    model.add(Dense(32)) 
    model.add(Dense(24))
    model.add(Dense(16))
    model.add(Dense(12, activation='softmax'))
    opt = Adam(learning_rate=0.01)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit(X[train], y[train],
                batch_size=25,epochs=50,
                verbose=1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

# Adding in Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(X_train)

## Convolutional Neural Network

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores = []
histories = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (128,128,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(12, activation = "softmax"))

    opt = Adam(learning_rate=0.01)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

    model.fit_generator(datagen.flow(X_train,y_train, batch_size=25),epochs = 50,
                                  validation_data = (X_test,y_test),verbose = 1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## Dense Neural Network

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_d = []
histories_d = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(200))
    model.add(Dense(100))
    model.add(Dense(64))
    model.add(Dense(32)) 
    model.add(Dense(24))
    model.add(Dense(16))
    model.add(Dense(12, activation='softmax'))

    opt = Adam(learning_rate=0.01)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

    model.fit_generator(datagen.flow(X_train,y_train, batch_size=25),epochs = 50,
                                  validation_data = (X_test,y_test),verbose = 1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## AlexNet

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_a = []
histories_a = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(128,128,3)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.25)),
    model.add(Dense(12, activation='softmax'))
    
    opt = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=25),epochs = 50,
                                  validation_data = (X_test,y_test),verbose = 1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## ResNet50

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_res = []
histories_res = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
    model.add(Dense(64))
    model.add(Dense(32)) 
    model.add(Dense(24))
    model.add(Dense(16))
    model.add(Dense(12, activation='softmax'))
    opt = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=25),epochs = 50,
                                  validation_data = (X_test,y_test),verbose = 1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

# Flower Classification

## Convolutional Neural Network

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores = []
histories = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (128,128,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(5, activation = "softmax"))

    opt = Adam(learning_rate=0.01)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit(X[train], y[train],
                batch_size=25,epochs=50,
                verbose=1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## Dense Neural Network

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_d = []
histories_d = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(200))
    model.add(Dense(100))
    model.add(Dense(64))
    model.add(Dense(32)) 
    model.add(Dense(24))
    model.add(Dense(16))
    model.add(Dense(5, activation='softmax'))
    opt = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

    history = model.fit(X[train], y[train],batch_size=25,epochs=50,verbose=1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## Alex Net

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_a = []
histories_a = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(128,128,3)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.25)),
    model.add(Dense(5, activation='softmax'))
    
    opt = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit(X[train], y[train],batch_size=25,epochs=50,verbose=1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## ResNet

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_res = []
histories_res = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
    model.add(Dense(64))
    model.add(Dense(32)) 
    model.add(Dense(24))
    model.add(Dense(16))
    model.add(Dense(5, activation='softmax'))
    opt = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit(X[train], y[train],batch_size=25,epochs=50,verbose=1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

# Adding in Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(X_train)

## Convolutional Neural Network

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores = []
histories = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (128,128,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same',activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(5, activation = "softmax"))

    opt = Adam(learning_rate=0.01)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=25),epochs = 50,
                                  validation_data = (X_test,y_test),verbose = 1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## Dense Neural Network

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_d = []
histories_d = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(200))
    model.add(Dense(100))
    model.add(Dense(64))
    model.add(Dense(32)) 
    model.add(Dense(24))
    model.add(Dense(16))
    model.add(Dense(5, activation='softmax'))
    opt = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

    history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=25),epochs = 50,
                                  validation_data = (X_test,y_test),verbose = 1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## AlexNet

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_a = []
histories_a = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(128,128,3)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.25)),
    model.add(Dense(5, activation='softmax'))
    
    opt = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=25),epochs = 50,
                                  validation_data = (X_test,y_test),verbose = 1)

    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

## ResNet50

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=1234)
cvscores_res = []
histories_res = []
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
    model.add(Dense(64))
    model.add(Dense(32)) 
    model.add(Dense(24))
    model.add(Dense(16))
    model.add(Dense(5, activation='softmax'))
    opt = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=25),epochs = 50,
                                  validation_data = (X_test,y_test),verbose = 1)
    
    scores = model.evaluate(X[test], y[test])

    print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    cvscores.append(scores[1] * 100)
    histories.append(history)

# Predictions

In [ ]:
from keras.preprocessing import image

In [ ]:
path = '../input/plant-seedlings-classification/test/0021e90e4.png'
img = image.load_img(path, target_size=(128, 128))
x = image.img_to_array(img)
x = x / 255
x = np.expand_dims(x, axis=0)
classes = model.predict(x)
print(classes)

In [ ]:
prob = []
for c in classes:
    for p in c:
        prob.append(p)

In [ ]:
index = prob.index(max(prob))

In [ ]:
index

In [ ]:
prob[index]

In [ ]:
path = '../input/tulip-picture'
img = image.load_img(path, target_size=(128, 128))
x = image.img_to_array(img)
x = x / 255
x = np.expand_dims(x, axis=0)
classes = model.predict(x)
print(classes)

In [ ]:
prob = []
for c in classes:
    for p in c:
        prob.append(p)

In [ ]:
index = prob.index(max(prob))

In [ ]:
index

In [ ]:
prob[index]